In [12]:
import pandas as pd
from pathlib import Path
import json
from pprint import pprint

setup_data_path = Path().resolve() / 'setup_data'

In [4]:
MODELS = ['gpt-4o-mini', 'gpt-4o'] # gpt-3.5-turbo-16k-0613
EDITIONS = [f'FINAL_{i}' for i in range(4)]
GRAPH_TYPE = 'central_agent_1'
TRIAL_NAME = 'final_run_1'
PROMPTING_METHOD = 'default'

log_paths = {edit : {model : Path().resolve() / 'logs' / edit / GRAPH_TYPE / model / PROMPTING_METHOD / TRIAL_NAME for model in MODELS} for edit in EDITIONS}

In [40]:
def process_toks(josn):
    in_toks = 0
    out_toks = 0
    for v in josn['final_vals']['messages']:
        if 'AIMessage' in v['id']:
            in_toks += v['kwargs']['response_metadata']['token_usage']['prompt_tokens']
            out_toks += v['kwargs']['response_metadata']['token_usage']['completion_tokens']
    return in_toks,out_toks
    

In [43]:
save = {}
for exp in EDITIONS:
    save[exp] = {}
    for model in log_paths[exp].keys():
        comp = {'input' : 0, 'output' : 0}
        for i in range(30):
            with open(log_paths[exp][model] / f'env_{i}.json') as f:
                data = json.load(f)
            inp, outp = process_toks(data)
            comp['input'] += inp
            comp['output'] += outp
        comp['input'] /= i+1
        comp['output'] /= i+1
        save[exp][model] = comp

In [52]:
save['FINAL_3']

{'gpt-4o-mini': {'input': 111686.6, 'output': 18816.233333333334},
 'gpt-4o': {'input': 16981.933333333334, 'output': 4122.6}}

In [53]:
comparis = {}

for e in save:
    comparis[e] = {'input' : 0, 'output' : 0}
    for v in save[e]['gpt-4o']:
        comparis[e][v] = save[e]['gpt-4o-mini'][v] / save[e]['gpt-4o'][v]

In [54]:
comparis

{'FINAL_0': {'input': 9.72785817883718, 'output': 4.376596045419565},
 'FINAL_1': {'input': 5.874085137839953, 'output': 4.249004742617787},
 'FINAL_2': {'input': 5.609337574298387, 'output': 4.634343167085594},
 'FINAL_3': {'input': 6.5767894507496205, 'output': 4.564166626239104}}